In [ ]:
# I import all the packages I need
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk

In [ ]:
# I upload the dataset related to the ratings that contains the rating (out of 5) by each user for some movies

ratings= pd.read_csv('https://raw.githubusercontent.com/ValentinaCoppi/IT-coding-project/main/ratings.csv')
ratings.head()

In [ ]:
# I upload the dataset that contains the title of the movies and their genre

movies= pd.read_csv('https://raw.githubusercontent.com/ValentinaCoppi/IT-coding-project/main/movies.csv')
movies.head()

My aim is to create a collaborative filtering based on the cosine distance between the users, in order to fill in the gaps of the rating for the unseen movies, and propose the user the best movies he may like.

In [ ]:
# I calculate the mean rating for each user
def meanbyuser(user):
    return ratings.groupby('userId')[['rating']].mean().rating[user]

In [ ]:
mean_rating= ratings.groupby('userId')[['rating']].mean()

In [ ]:
#I change the names at the original dataset and the userId column

x = ratings.copy()
y = ratings['userId']


In [ ]:
# I need to transform the training dataset in a matrix that has users as row index and movie as column index, and fill it in with the ratings

df_ratings = x.pivot(index='userId', columns= 'movieId', values='rating')

In [ ]:
# I have a look at the dataset
df_ratings

In [ ]:
# I create a copy of the dataset, in order to modify it without causing problems
df_ratings0= df_ratings.copy()

In [ ]:
# I want to normalize all the ratings, subtracting the mean value for each user, in order to make the ratings more fair

for i in range(1,len(mean_rating['rating']+1)):
    df_ratings0.loc[i,:]=np.array(df_ratings0.loc[i,:])-mean_rating['rating'][i] 


In [ ]:
# This is the new dataframe with the normalized ratings
df_ratings0

In [ ]:
# I want that NaN are filled with 0, or I can't work on the matrix
# For doing that I create a copy of df_ratings0 

df_ratings1 = df_ratings0.copy().fillna(0)
df_ratings1.head()

In [ ]:
# I calculate the cosine similarity of the users and I create a dataframe that contains it

similarity_matrix = cosine_similarity(df_ratings1, df_ratings1)
similarity_matrix_df = pd.DataFrame(similarity_matrix, index=df_ratings.index, columns=df_ratings.index)

In [ ]:
# Let's have a look at the similarity matrix
similarity_matrix_df.head()

Now, all the main elements have been created, so I can write the main function of the program, the one that actually recommand the movies.
The recommandation system is user based, so the funtion requires to isert one of the UserId and will give back both the list of the movies seen by the User chosen and the first 20 movies per possible rating, basing on similar users ratings.

In [ ]:
def Recommand(picked_userid):   
    # I remove picked UserId from the candidate list in the similarity matrix previously created
    similarity_matrix_df.drop(index=picked_userid, inplace=False)
   
    # I decide that the number of similar user that I want consider is 10
    n = 10
    # Anyway I don't want users that are not similar at all, so I put a User similarity threashold
    user_similarity_threshold = 0.09
    # Here, I get top n similar users that exceed the threshold
    similar_users = similarity_matrix_df[similarity_matrix_df[picked_userid]>user_similarity_threshold][picked_userid].sort_values(ascending=False)[:n]

    #I make a dataframe with the movies that the target user has watched (I actually make two, one with the normalized data and one with the real one)
    picked_userid_watched = df_ratings0[df_ratings0.index == picked_userid].dropna(axis=1, how='all')
    picked_userid_watched1 = df_ratings[df_ratings.index == picked_userid].dropna(axis=1, how='all')
    #I make a dataframe with the movies that similar users watched and I remove movies that none of the similar users have watched (drop the columns with all NA)
    similar_user_movies = df_ratings0[df_ratings0.index.isin(similar_users.index)].dropna(axis=1, how='all')
    #Now, I remove the watched movies by the chosen User from the movie list. 
    #In this way I got a dataframe with all the movies that the picked user did't watch, but some of the similar users did 
    similar_user_movies.drop(picked_userid_watched.columns,axis=1, inplace=True, errors='ignore')
    
    #I create a dictionary to store item scores
    item_score = {}
    #I loop through movies
    for i in similar_user_movies.columns:
      # I get the ratings for movie i
        movie_rating = similar_user_movies[i]
      # I create a variable to store the score
        total = 0
      # I create a variable to store the number of scores
        count = 0
      # I loop through similar users
        for u in similar_users.index:
        # If the movie has rating
            if pd.isna(movie_rating[u]) == False:
          # Score is the sum of user similarity score multiply by the movie rating, because I want that more similar users weight more in the movies score
              score = similar_users[u] * movie_rating[u]
          # I add the score to the total score for the movie so far
              total += score
          # I increase the count
              count +=1
      # Now, I get the average score for the item and I add back the mean score for the user picked
        item_score[i] = total / count+ meanbyuser(picked_userid)
    # I convert the dictionary to pandas dataframe and I format the avarage rate with 2 decimal digits
    item_score = pd.DataFrame(item_score.items(), columns=['movieId', 'movie_score'])
    item_score['movie_score'] = item_score['movie_score'].apply(lambda x: float("{:.2f}".format(x)))
    # I sort the movies by score
    ranked_item_score = item_score.sort_values(by='movie_score', ascending=False)
    # I create a dataframe with the movieId, the estimated rate, the movie title, and the movie genre that is the recommandation datatset
    best_movies= pd.merge( ranked_item_score, movies, on= 'movieId')
    # I create a dataframe with movieId, rate, movie title and genre, that is the the dataframe with the movies watched
    picked_userid_watched1= pd.merge(picked_userid_watched1.T, movies, on= 'movieId')
    # I sort the previous dataset by rate
    picked_userid_watched2=picked_userid_watched1.sort_values(by=[picked_userid], ascending=False)
    # I rename one of the columns because it create some problems in the execution, since it wasn't a string
    picked_userid_watched2.rename(columns = {picked_userid:'Rate'}, inplace = True)
    # I select top m movies: I decided to recommand just 20 movies
    m = 20
    # I write the first 20 recommanded movies in an excel file and even the first 20 watched movies by rate in another excel file (I decided to take 20 even in this case, just to get the preferred by the user)
    best_movies.head(m).to_excel("output.xlsx")
    picked_userid_watched2.head(m).to_excel('watched.xlsx')
    # I ask to return them in order to have a look when I just call the function
    return best_movies.head(m)
    return picked_userid_watched2.head(m)

I decided to create a simple GUI in which you can insert the UserId and it has two buttons: one have a look at the movies the user liked the most (in order that you can make an idea of the user preferences) and the other shows you the recommanded movies for that User.
If you insert a non valid userId a messagebox is shown, telling you that your userId is invalid, if you close it you can try again. 
The dataframe are shown in toplevel windows that can be closed and you can start from the beginning without exectuing the code again.


The allowed userId are the numbers from 1 to 610

In [ ]:
# I create the main window with a title,a size and a beckground color
window = tk.Tk()
window.title("Choose UserId")
window.geometry('600x400')
window.configure(bg='#333333')

# I define the function that checks if the userId is valid or not. If it is valid it open the recommand movies window, if it's not it opens the error messagebox
def login():
    username = range(1,len(df_ratings)+1)
    try:
        if int(username_entry.get()) in username:
            open_secondary_window()
    except:
        messagebox.showinfo(title="Error", message="Invalid UserId.")
        
 # I define the function that checks if the userId is valid or not. If it is valid it open the watched movies window, if it's not it opens the error messagebox      
def look():
    username = range(1,len(df_ratings)+1)
    try:
        if int(username_entry.get()) in username:
            open_third_window()
    except:
        messagebox.showinfo(title="Error", message="Invalid UserId.")

# I create a frame with a beckground color

frame = tk.Frame(bg='#333333')

# This is the funtion that creates the secondary window with the recommandations    
def open_secondary_window():
    # Create secondary window with a title and a beckground color
    secondary_window = tk.Toplevel()
    secondary_window.title("Reccomanded movies")
    secondary_window.config(bg='#333333')
    
    # I call the Recommand function with argument the UserId I entered in the main window
    Recommand(int(username_entry.get()))
    # I write the excel file as pandas dataframe, with only the necessary columns
    df= pd.read_excel("output.xlsx", usecols= 'C, D,E')
    # I extract number of rows and columns from the dataframe
    n_rows = df.shape[0]
    n_cols = df.shape[1]

    # I extract columns from the data and I create a text widget with some background color
    # I didn't make a loop, but I repeated the code because I wanted to have different width of the columns
    column_names = df.columns
    i=0
    text = tk.Text(secondary_window, width=15, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=0)
    text.insert(tk.INSERT, 'Possible rate')

    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=1)
    text.insert(tk.INSERT, 'Title')
    
    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=2)
    text.insert(tk.INSERT, 'Genre')

    # NOw, I add all the other rows into the grid, mantaining the same width as the title
    for i in range(n_rows):

        text = tk.Text(secondary_window, width=15, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=0)
        text.insert(tk.INSERT, df.loc[i][0])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=1)
        text.insert(tk.INSERT, df.loc[i][1])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=2)
        text.insert(tk.INSERT, df.loc[i][2])
      
    # This command is needed because when the secondary window is open, you cannot interact with the main one
    secondary_window.grab_set() 
    
# This function is exactly the same as the previous one, with the only difference that the file opened is the watched movies one
# I do not repeat the comments, since they are identical   
def open_third_window():

    secondary_window = tk.Toplevel()
    secondary_window.title("Watched movies")
    secondary_window.config(bg='#333333')
    
    Recommand(int(username_entry.get()))
    df= pd.read_excel("watched.xlsx", usecols= 'C, D,E')
    
    n_rows = df.shape[0]
    n_cols = df.shape[1]

    column_names = df.columns
    i=0
    text = tk.Text(secondary_window, width=15, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=0)
    text.insert(tk.INSERT, 'Possible rate')

    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=1)
    text.insert(tk.INSERT, 'Title')
    
    text = tk.Text(secondary_window, width=40, height=1, bg='#00CCCC', font=("Arial", 16))
    text.grid(row=i,column=2)
    text.insert(tk.INSERT, 'Genre')
    

    for i in range(n_rows):

        text = tk.Text(secondary_window, width=15, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=0)
        text.insert(tk.INSERT, df.loc[i][0])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=1)
        text.insert(tk.INSERT, df.loc[i][1])

        text = tk.Text(secondary_window, width=40, height=1, bg='#333333', fg="#FFFFFF", font=("Arial", 16))
        text.grid(row=i+1,column=2)
        text.insert(tk.INSERT, df.loc[i][2])
    
    
    secondary_window.grab_set()  # Modal.

# Here, I create the widgets
# Label with the request with a certain color and with a certain font and size
login_label = tk.Label(
    frame, text="  Choose the UserId desired", bg='#333333', fg='#00CCCC', font=("Arial", 30))
# Label with the username with a certain color and with a certain font and size
username_label = tk.Label(
    frame, text="Username", bg='#333333', fg="#FFFFFF", font=("Arial", 16))
# Here you can enter the chosen UserId
username_entry= tk.Entry(frame, font=("Arial", 16))
# I create the button that call the login function
login_button = tk.Button(
    frame, text="Recommand", bg="#00CCCC", fg="#FFFFFF", font=("Arial", 16), command=login)
# I create the button that call the look funtion 
liked_button = tk.Button(
    frame, text="My movies", bg="#00CCCC", fg="#FFFFFF", font=("Arial", 16), command=look)

# I place widgets on the screen
login_label.grid(row=0, column=0, columnspan=3, sticky="news", pady=40)
username_label.grid(row=1, column=0)
username_entry.grid(row=1, column=1,columnspan=2, pady=20)
login_button.grid(row=3, column=1, columnspan=1, pady=10)
liked_button.grid(row=4, column=1, columnspan=1, pady=10)

# I show my GUI
frame.pack()

window.mainloop()